In [12]:
import json
import spacy
import markovify
import re

In this project, we'll be working with a JSON from Scryfall. 
They provide a JSON with all their data here: https://scryfall.com/docs/api/bulk-data
We're using the all English cards datset.

First, we'll load in the data using python's native JSON module.

In [2]:
with open( 'all_english_june_2021.json', 'r', encoding = 'utf-8') as input:
    cards_json = json.load(input)


Then, we can add unique flavor texts to a giant list. Since we only want 'real' in-world flavor texts, we'll ignore silver-bordered cards

In [15]:
flavor_text_list = []
for card in cards_json:
    #we don't want silver-bordered cards in here
    if card.get('border_color') != "silver":
        flavor_text = card.get('flavor_text')
        #ignore noneTypes
        if type(flavor_text) != type(None):
            #make flavor texts single line, and get rid of quotes and asterisks (used to indicate italics) Slashes indicate verses, and we don't want those either
            flavor_text = flavor_text.replace("\n", "").replace('"', "").replace("*", "").replace(" / ", " ")
            #replace the extraneous slashes
            flavor_text = re.sub('\D/\D', '', flavor_text)
            if flavor_text not in flavor_text_list:
                flavor_text_list.append(flavor_text)

    corpus_string = "\n".join(flavor_text_list)

In [18]:
nlp = spacy.load("en_core_web_lg")

class POSifiedText(markovify.Text):
    def word_split(self, sentence):
        return ["::".join((word.orth_, word.pos_)) for word in nlp(sentence)]

    def word_join(self, words):
        sentence = " ".join(word.split("::")[0] for word in words)
        return sentence

text_model = POSifiedText(corpus_string, well_formed = False)

In [66]:
def space_correctly(input):
    spaced = input.replace(" '", "'").replace(" .", ".").replace(" ?", "?").replace(" ,", ",").replace(" ;", ";").replace(" !", "!").replace(" n't", "n't").replace(" - ", "-")
    split_on_lines = spaced.split("\n")
    return split_on_lines[0]

In [97]:
# Print five randomly-generated sentences
print("Random sentences:")
for i in range(5):
    print(space_correctly(text_model.make_sentence()))

print("\nShort sentences:")
# Print three randomly-generated sentences of no more than 280 characters
for i in range(5):
    print(space_correctly(text_model.make_short_sentence(280)))

# Print randomly-generated sentences that start with a certain word
word = "Gideon"
print(f"\nSentences that start with {word}:")
for i in range(5):
    print(space_correctly(text_model.make_sentence_with_start(word, strict=False)))

Random sentences:
Their wisdom I crave, but for reasons known only to those who make such claims see only what we meant when we stand together.—Numa, Joraga hunter 
The king would die for nothing, you fellas are going to be, but they care only for the path to unity is forged in confidence.
I was counting on it when stricken.—Lim-Dûl, the might of the Swarm begins.
It was the worst of the Weatherlight as his longing, their story lost to them.—Vy Covalt, Mirran resistance 
We strike without fear.

Short sentences:
Dark schemes cut down my angel I only started my fall.—Crovax 
They smother their prey and then there was night.
Clouds are the markers of its existence.
I have some teeth.—Aeschel, royal archivist 
By the tongues of a rakshasa is soft, its commander wounded, it lives by one.

Sentences that start with Gideon:
Gideon's right  we all need someone to watch them reassemble one another.
Gideon — he has great form.—Liliana Vess 
Gideon never believed in a weapon, perfecting its visi

In [57]:
def brute_force_sentence_about(topic, tries = 1000):
    lowercase_topic = topic.lower()
    i = 0
    while i <= tries:
        sentence = space_correctly(text_model.make_sentence())
        if lowercase_topic in sentence.lower():
            return sentence
        else:
            i+=1
    return f"No sentence about {topic} could be generated in {tries} attempts."


In [81]:
# Print randomly-generated sentences that include with a certain word
word = "Phyrexia"
print(f"\nSentences about {word}:")
for i in range(5):
    print(brute_force_sentence_about(word))


Sentences about —:
But the other side.—Telim'Tor 
This is necessity.—Kaervek 
Could they be following surges in crystalline energy?—Naireh, Ketria elementalist 
What walls?—Jaya Ballard, Task Mage 
We respect all lifeforms, but the venom glands, then we'll build you a Rat to madness tease Why ev'n a Rat to madness tease Why ev'n a Rat may plague you... — Samuel Butler 


In [83]:
def generate_quote(tries = 1000):
    return brute_force_sentence_about("—", 1000)


In [95]:
# Print quotes
print(f"\nQuotes:")
for i in range(5):
    print(generate_quote())


Quotes:
Despite their stake in the form you may transcend your situation.—Gerda Äagesdotter, Archmage of the pariah is the most respected by storm and wished to hunt the Returned and the hot summer sun and trumpet their passing.
I'm smart enough to kill a lot smaller and its father, I can look down on them.—Dierk, geistmage 
She memorized every arcane text in the fight.—Observations of the living.—Thalia, Knight-Cathar 
We've survived there so much that they existed at all.—Ganedor, loxodon surgeon 
Under Yavimaya's other, more quickly than age.—Sensei Golden-Tail 


In [98]:
print(type(text_model))

<class '__main__.POSifiedText'>
